In [ ]:
import pandas as pd
from datetime import datetime
from stackapi import StackAPI

In [ ]:
import os
directory_path = r'C:\Users\mkommaraju\Documents\Work\ML\stackoverflow'
os.chdir(directory_path)

In [ ]:
today_date = datetime.today().strftime('%Y-%m-%d')
start_date= pd.date_range('2019-01-01',today_date , freq='MS').strftime('%Y-%m-%d') # first of the month
end_date  = pd.date_range('2019-01-01',today_date , freq='M').strftime('%Y-%m-%d')  # last of the month (ends at previous month)
N = len(end_date) # not start_date, which has one additional month
tag_list = ['paypal','stripe-payments']

In [ ]:
import urllib3
urllib3.disable_warnings()
# https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
# I get warning because in stackapi.py, line 183, I added verify=False
# C:\Users\mkommaraju\AppData\Local\Continuum\anaconda3\lib\site-packages\stackapi\stackapi.py

In [ ]:
SITE = StackAPI('stackoverflow')

questions_items_full_list=[]
for tag in tag_list:
    for i in range(N):
        print("'{}' tag, start={}, end={}".format(tag, start_date[i], end_date[i]) )
        questions = SITE.fetch('questions', 
                               fromdate=datetime.strptime(start_date[i], '%Y-%m-%d'), 
                               todate=datetime.strptime(end_date[i], '%Y-%m-%d'), 
                               tagged=tag)
        questions_items = questions.get('items')
        questions_items[1]
        for q in questions_items:
            q.update({'search_tag':tag})
        questions_items_full_list = questions_items_full_list + questions_items

df = pd.DataFrame(questions_items_full_list)
df["creation_date_str"] = [datetime.fromtimestamp(t).strftime('%Y-%m-%d') for t in df.creation_date]
df["is_answered_flag"] = [1 if f==True else 0 for f in df.is_answered]

In [ ]:
import html.parser; 
html.parser.HTMLParser().unescape(query)
df["title_clean"] = [html.parser.HTMLParser().unescape(f) for f in df.title]

df.to_csv('stackoverflow.csv',index=False)

In [ ]:
from urllib.parse import urlparse
from googlesearch import search 
# to search 
#query = df.search_tag[0] + ' ' + df.title_clean[0]
#query = 'paypal What is the difference between Paypal preapproved payments and automatic payments'

In [ ]:
#N_questions = 5
question_id_list = []
result_url_list = []
result_url_short_list = []
search_tag_list = []

N_questions = df.shape[0]
for i in range(N_questions):
    query = df.search_tag[i] + ' ' + df.title_clean[i]
    search_result = search(query, tld="com", num=10, stop=10, pause=1)
    search_result_list = list(search_result)
    short_result_url = []
    print(i)
    for res in search_result_list: 
        search_tag_list.append(df.search_tag[i])
        question_id_list.append(df.question_id[i])
        result_url_list.append(res)
        result_url_short_list.append(urlparse(res).netloc)

df_google_search = pd.DataFrame({'search_tag':search_tag_list,
                                 'question_id':question_id_list,
                                 'result_url':result_url_list,
                                 'result_url_short':result_url_short_list
                                 })
df_google_search.reset_index(drop=False, inplace=True)
df_google_search['search_rank'] = df_google_search.groupby(['search_tag','question_id'])['index'].rank(method='first')

df_google_search.to_csv('df_google_search.csv',index=False)

In [ ]:
df = pd.read_csv('stackoverflow.csv')
df.columns
df.tags
df_tags_list = []
df_question_id_list = []
for i in range(df.shape[0]):
    df_tags = df.tags[i].strip('[]').replace(' ','').replace('\'','').split(',')
    q = df.question_id[i]
    for t in df_tags:
        df_tags_list.append(t)
        df_question_id_list.append(q)
df_tags_map = pd.DataFrame({'question_id':df_question_id_list,
                            'tag_id':df_tags_list
                            })
df_tags_questions = df_tags_map.merge(df, left_on='question_id', right_on='question_id')
df_tags_questions.to_csv('df_tags_questions.csv', index=False)